# Grid dispatch optimization by linear programming

**⚠️ Warning!** This notebook is out of date and not maintained anymore. Please use the `run_coal_single.py` and `run_coal_all.sh` scripts to run the model and access the latest features. If you encounter any problems, please get in touch with our customer support.


## Imports


In [ ]:
# Add parent dir to the path, so we can load the module

import os
import sys
from copy import deepcopy

sys.path.append(os.path.dirname(os.path.abspath("")))

from energy_insights.execution_utils import (
    get_entsoe_loader,
    get_extrapolator,
    get_pecd_loader,
    optimize_runs,
)
from energy_insights.region import *
from energy_insights.scenarios.czech_coal import (
    cz_2025_basic,
    cz_2028_basic,
    cz_2028_progressive,
    global_adjustments,
    make_pessimistic_scenario,
    make_run,
)


## Config


In [ ]:
common_year = 2018
# ENTSO-E has only data back to 2010, we need a fallback year for the load.
# TODO: Try ENTSO-E 2020 & PECD 2009 -- leapness mismatch.
entsoe_year = 2018
# NOTE: There's a mismatch between the weather years (common_year and
# entsoe_year) and PECD year because we only have weather starting 2019,
# but the PECD dataset ends in 2016.
pecd_year = 1985
aggregation_level = "coarse"

entsoe_loader = get_entsoe_loader("../data")
pecd_loader = get_pecd_loader("../data")
extrapolator = get_extrapolator(entsoe_loader, pecd_loader)

runs = [
    make_run(
        "precerpavacky",
        scenarios=[
            {
                "name": "2025-cheap-ets",
                "year": 2025,
                "adjustments": cz_2025_basic,
                "global_adjustments": global_adjustments,
                "input_costs": "2025-cheap-ets",
            },
            {
                "name": "2025-expensive-ets",
                "year": 2025,
                "adjustments": cz_2025_basic,
                "global_adjustments": global_adjustments,
                # "optimize_capex": False,
            },
            {
                "name": "2028-slow",
                "year": 2028,
                "adjustments": cz_2028_basic,
                "global_adjustments": global_adjustments,
            },
            {
                "name": "2028-advanced",
                "year": 2028,
                "adjustments": cz_2028_advanced,
                "global_adjustments": global_adjustments,
            },
        ],
        aggregation_level=aggregation_level,
        common_year=common_year,
        entsoe_year=entsoe_year,
        pecd_year=pecd_year,
        optimize_capex=True,
        # Optimize coal capacity in all regions.
        optimize_coal="all",
        optimize_heat=True,
        optimize_ramp_up_costs=True,
        root_dir="..",
    ),
]


### Pessimistic scenarios

A single scenario with pessimistic assumptions:

- pick the minimal installed capacities among 2025 and 2028 values,
- 20% lower interconnection capacities across the board.


In [ ]:
# def minimize_sources(sources1: dict, sources2: dict) -> None:
#     for source, source_spec in sources1.items():
#         capacity_mw = min(
#             source_spec["capacity_mw"],
#             sources2.get(source, {}).get("capacity_mw", 0)
#         )
#         source_spec["capacity_mw"] = capacity_mw
#         # Respect capex optimization.
#         source_spec["min_capacity_mw"] = (
#             0 if source_spec.get("min_capacity_mw", 0) == 0
#             else capacity_mw
#         )


# def scale_intercon_capacities(scenario: dict, factor: float) -> dict:
#     interconnectors: InterconnectorsDict = scenario["interconnectors"]

#     for destination_map in interconnectors.values():
#         for link_spec in destination_map.values():
#             link_spec["capacity_mw"] *= factor

#     return scenario


# scenario_2025, scenario_2028, _ = runs[0]["scenarios"]
# scenario_min = deepcopy(scenario_2028)
# scenario_min["name"] = "2028-pessi"

# for country, spec_min in scenario_min["countries"].items():
#     spec_2025 = scenario_2025["countries"][country]

#     # Reconcile basic sources.
#     minimize_sources(spec_min["basic_sources"], spec_2025["basic_sources"])

#     # Reconcile flexible sources.
#     minimize_sources(spec_min["flexible_sources"], spec_2025["flexible_sources"])

# # Reduce interconnection capacities left by 20% (compared to 2025).
# scale_intercon_capacities(scenario_min, .8)

# # Increase Czech electricity demand by 10%.
# scenario_min["countries"][CZECHIA]["load_factors"]["load_base"] = 1.0

# # Construct a run by copying the base one.
# run_min = deepcopy(runs[0])
# run_min["config"]["analysis_name"] = "coaldown-pessi"
# run_min["scenarios"] = [scenario_min]

# # runs.insert(0, run_min)
# runs = [run_min]


## Execution


In [ ]:
optimize_runs(
    runs,
    entsoe_loader=entsoe_loader,
    pecd_loader=pecd_loader
)
